In [ ]:
from __future__ import absolute_import
from __future__ import print_function
import os


os.environ['KERAS_BACKEND'] = 'theano'
os.environ['THEANO_FLAGS']='mode=FAST_RUN,device=cuda0,floatX=float32,optimizer=None'


import keras
import keras.models as models

from keras.models import Sequential, Model
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers import BatchNormalization,Input
from keras.layers.recurrent import SimpleRNN, LSTM
from keras.layers.convolutional import Convolution2D
from keras.optimizers import SGD, Adam, RMSprop
import sklearn.metrics as metrics

from keras.callbacks import ModelCheckpoint

import cv2
import numpy as np
import json

# from Dan Does Data VLOG
import math
import h5py
import glob
from tqdm import tqdm
import scipy
from scipy import misc

import matplotlib.pyplot as plt
plt.ion()
%matplotlib inline

ndata = 0
imgsize = 128
# frame size
nrows = 128
ncols = 128

In [ ]:
imgs = np.load('data/20190106-213038-images.npz')['arr_0']
targets = np.load('data/20190106-213038-steer.npz')['arr_0']

In [ ]:
idx = 0
imgplot = plt.imshow(imgs[idx,0,:].astype(np.float32), cmap="gray")
print(f'steer: {targets[idx]}')

In [ ]:
# Shuffle images and steering targets
idx = np.arange(0,imgs.shape[0])
idx = np.random.permutation(idx)
imgs = imgs[idx,:,:,:]
targets = targets[idx]

In [ ]:
# load the model:
model = Sequential()
with open('autopilot_basic_model.json') as model_file:
    model = models.model_from_json(model_file.read())


In [ ]:
# checkpoint
filepath="weights/weights.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

In [ ]:
adam = Adam(lr=0.0001)
model.compile(loss='mse',
              optimizer=adam,
              metrics=['mse','accuracy'])

nb_epoch = 25
batch_size = 64

model.fit(imgs, targets[:]*2-1, callbacks=callbacks_list,
	batch_size =batch_size, nb_epoch=nb_epoch, verbose=1,
	validation_split=0.1,shuffle=True)

In [ ]:
model.save_weights('weights/model_basic_weight.hdf5')